# Sentiment
(Codice autoconsistente)


In [ ]:
# Installazione delle dipendenze
!pip install pandas torch tqdm scikit-learn transformers feel-it

In [3]:
# Importazione delle librerie necessarie
import pandas as pd
import torch
from tqdm import tqdm
from sklearn.metrics import classification_report
from torch import nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
from feel_it import SentimentClassifier
from sklearn.metrics import classification_report

## 1) Italian BERT Sentiment model
https://huggingface.co/neuraly/bert-base-italian-cased-sentiment 

Questo modello esegue l'analisi del sentiment sulle frasi italiane. È stato addestrato a partire da un'istanza di bert-base-italian-cased, e messo a punto su un dataset italiano di tweets, raggiungendo su quest'ultimo l'82% di precisione.

In [33]:
# Caricamento del dataset: estraggo il DataFrame dal file excel
data_path = '../POI con recensioni/POI.xlsx'    # Aggiorna con il percorso corretto del tuo file
df = pd.read_excel(data_path)

In [35]:
# Visualizzazione del DataFrame
df

,id,POI,rating,recensione
0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume..."
1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...
2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale è ...
3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...
4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta é il principale mo...
...,...,...,...,...
5330,663c8b16ec1e0d04a559c75e,Cacio Pepe e,5,Ci siamo trovati di passaggio dopo aver visita...
5331,663c8b16ec1e0d04a559c75f,Annette Jolie,5,"Girando per Assisi ,ho trovato un negozio fina..."
5332,663c8b16ec1e0d04a559c75f,Annette Jolie,5,A pochi passi da Piazza del Comune abbiamo sco...
5333,663c8b16ec1e0d04a559c760,Sorella Luna,5,Negozio posto in una via secondaria ma dove va...


In [36]:
# Caricamento del tokenizzatore e del modello BERT
tokenizer = AutoTokenizer.from_pretrained("neuraly/bert-base-italian-cased-sentiment")
bert_model = AutoModelForSequenceClassification.from_pretrained("neuraly/bert-base-italian-cased-sentiment")
bert_model.eval()  # Imposta il modello in modalità valutazione

c:\Users\franc\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32102, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

1. Estrae la recensione dalla colonna recensione.
2. Codifica la recensione utilizzando un tokenizer, parte di un modello di linguaggio preaddestrato.
3. Trasforma la sequenza codificata in un tensore e lo prepara per l'input al modello.
4. Passa il tensore al modello per ottenere i logit, che sono le uscite del modello prima di applicare la funzione di attivazione.
5. Applica una funzione di softmax sui logit per ottenere le probabilità associate a ciascuna classe (probabilità negative, neutre e positive).
6. Memorizza le probabilità negative, neutre e positive ottenute nella tabella DataFrame df.
7. Determina la classe di sentimento prevista utilizzando la probabilità massima tra negative, neutre e positive.
8. Memorizza la classe di sentimento prevista nella tabella DataFrame df.

In [37]:
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    # Itera attraverso ogni riga del DataFrame 'df', mostrando una barra di progresso con tqdm
    sentence = row.recensione

    # Tokenizza la frase della recensione usando il tokenizer specificato
    input_ids = tokenizer.encode(sentence, add_special_tokens=True, truncation=True, max_length=512)

    # Crea un tensore, usa .cuda() per trasferire il tensore alla GPU se disponibile
    tensor = torch.tensor(input_ids).long().unsqueeze(0)
    if torch.cuda.is_available():
        tensor = tensor.cuda()
        bert_model.cuda()

    # Chiama il modello BERT e ottiene i logits (uscite grezze prima dell'applicazione della funzione softmax)
    with torch.no_grad():
        logits = bert_model(tensor)
    
    # Rimuove la dimensione fittizia del batch dai logits
    logits = logits.logits.squeeze(0)

    # Applica la funzione softmax per ottenere le probabilità
    proba = nn.functional.softmax(logits, dim=0)

    # Estrae le probabilità per le classi negativo, neutro e positivo
    negative, neutral, positive = proba

    # Assegna le probabilità alle colonne 'negative', 'neutral' e 'positive' nel DataFrame
    df.loc[index, 'negative'] = proba[0].item()
    df.loc[index, 'neutral'] = proba[1].item()
    df.loc[index, 'positive'] = proba[2].item()


100%|██████████| 5335/5335 [04:56<00:00, 18.00it/s]


Il "tokenizer" è una componente fondamentale quando si lavora con modelli di linguaggio naturale (NLP). La sua funzione principale è quella di convertire il testo in una forma che il modello possa comprendere. Questo processo coinvolge solitamente la suddivisione del testo in parole o sotto-unità semantiche (come i token) e la successiva conversione di questi token in numeri, che possono essere utilizzati come input per il modello. Questo processo di tokenizzazione può anche includere altre operazioni, come la rimozione della punteggiatura, la gestione delle maiuscole e minuscole, e altro ancora.

Il "tensore", d'altra parte, è una struttura dati fondamentale nelle librerie di calcolo numerico, come TensorFlow o PyTorch, utilizzate per l'implementazione di modelli di machine learning e deep learning. In termini semplici, un tensore può essere considerato come un array multidimensionale, che può essere un vettore (1D), una matrice (2D) o una struttura multidimensionale di ordine superiore. I tensori sono essenziali poiché consentono di rappresentare i dati e le operazioni matematiche eseguite sui dati in modo efficiente e parallelo, specialmente quando si lavora con grandi quantità di dati.

In questo contesto il "tokenizer" viene utilizzato per convertire il testo delle recensioni in sequenze di token numerici, mentre il "tensore" viene utilizzato per rappresentare queste sequenze di token in una forma che il modello di machine learning possa elaborare.

In [38]:
df.head()

,id,POI,rating,recensione,negative,neutral,positive
0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume...",0.001215,0.010892,0.987893
1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...,0.000190,0.004603,0.995207
2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale è ...,0.000429,0.001524,0.998047
3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...,0.001172,0.004624,0.994203
4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta é il principale mo...,0.000578,0.051220,0.948202


In [39]:
# Determino l'etichetta del sentiment sulla base della probabilità più alta
df['BERT'] = df[['negative', 'positive', 'neutral']].idxmax(axis=1)


In [40]:
df.head()

,id,POI,rating,recensione,negative,neutral,positive,BERT
0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume...",0.001215,0.010892,0.987893,positive
1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...,0.000190,0.004603,0.995207,positive
2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale è ...,0.000429,0.001524,0.998047,positive
3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...,0.001172,0.004624,0.994203,positive
4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta é il principale mo...,0.000578,0.051220,0.948202,positive


In [41]:
df.to_excel('../sentiment/BERT.xlsx', index=False)

In [42]:
df = pd.read_excel('../POI con recensioni/POI.xlsx')
df.rating.value_counts()

rating
5    2637
4    1551
3     710
1     249
2     188
Name: count, dtype: int64

In [44]:
df = pd.read_excel('../sentiment/BERT.xlsx')
df.BERT.value_counts()

BERT
positive    4228
neutral      584
negative     523
Name: count, dtype: int64

In [45]:
df.to_csv("../sentiment/BERT_value_counts.csv", index=False, sep='\t')

### Calcolo dell'Accuracy

In [5]:
df = pd.read_excel('../sentiment/BERT.xlsx')

In [47]:
# Aggiunta di una colonna GS relativa a cosa ci si aspetta
df['GS']=0
df.loc[df['rating'] >= 4, 'GS']  = 'positive'
df.loc[df['rating'] == 3, 'GS']  = 'neutral'
df.loc[df['rating'] <= 2, 'GS']  = 'negative'
df

,id,POI,rating,recensione,negative,neutral,positive,BERT,GS
0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume...",0.001215,0.010892,0.987893,positive,positive
1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...,0.000190,0.004603,0.995207,positive,positive
2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale è ...,0.000429,0.001524,0.998047,positive,positive
3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...,0.001172,0.004624,0.994203,positive,positive
4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta é il principale mo...,0.000578,0.051220,0.948202,positive,positive
...,...,...,...,...,...,...,...,...,...
5330,663c8b16ec1e0d04a559c75e,Cacio Pepe e,5,Ci siamo trovati di passaggio dopo aver visita...,0.000208,0.001506,0.998286,positive,positive
5331,663c8b16ec1e0d04a559c75f,Annette Jolie,5,"Girando per Assisi ,ho trovato un negozio fina...",0.000223,0.001533,0.998243,positive,positive
5332,663c8b16ec1e0d04a559c75f,Annette Jolie,5,A pochi passi da Piazza del Comune abbiamo sco...,0.000134,0.001653,0.998213,positive,positive
5333,663c8b16ec1e0d04a559c760,Sorella Luna,5,Negozio posto in una via secondaria ma dove va...,0.000364,0.003089,0.996547,positive,positive


In [48]:
print(classification_report(df['GS'], df['BERT']))

              precision    recall  f1-score   support

    negative       0.62      0.74      0.67       437
     neutral       0.32      0.26      0.29       710
    positive       0.88      0.89      0.89      4188

    accuracy                           0.80      5335
   macro avg       0.61      0.63      0.62      5335
weighted avg       0.79      0.80      0.79      5335



In [49]:
# Calcola il classification report
report = classification_report(df['GS'], df['BERT'], output_dict=True)

# Creare un DataFrame pandas dal classification report
df_report = pd.DataFrame(report).transpose()

# Salvare il DataFrame come file Excel
excel_filename = '../sentiment/BERT_classification_report.xlsx'
df_report.to_excel(excel_filename, index=True)

## 2) FEEL - IT
https://huggingface.co/MilaNLProc/feel-it-italian-sentiment?text=Mi+piaci.+Ti+amo

Il modello Feel-it-Italian-Sentiment esegue l'analisi del sentiment sull'italiano. Viene utilizzato il modello UmBERTo sul nuovo set di dati (cioè FEEL-IT) ottenendo prestazioni all'avanguardia su diversi corpus di riferimento.

I dati sono stati raccolti annotando tweet di un’ampia gamma di argomenti. In totale, ci sono 2037 tweet annotati con un'etichetta emozionale.

PAPER: https://aclanthology.org/2021.wassa-1.8.pdf

In [2]:
# Carica il tokenizer e il modello
tokenizer = AutoTokenizer.from_pretrained("MilaNLProc/feel-it-italian-sentiment")
feelit_model = AutoModelForSequenceClassification.from_pretrained("MilaNLProc/feel-it-italian-sentiment")
feelit_model.eval()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
feelit_model.to(device)

c:\Users\franc\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


CamembertForSequenceClassification(
  (roberta): CamembertModel(
    (embeddings): CamembertEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): CamembertEncoder(
      (layer): ModuleList(
        (0-11): 12 x CamembertLayer(
          (attention): CamembertAttention(
            (self): CamembertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): CamembertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=Tru

In [3]:
def get_sentiment_probabilities(sentences, batch_size=32):
    encodings = tokenizer(sentences, truncation=True, padding=True, max_length=500, return_tensors='pt')
    input_ids = encodings['input_ids'].to(device)
    attention_mask = encodings['attention_mask'].to(device)

    with torch.no_grad():
        outputs = feelit_model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = F.softmax(logits, dim=-1).cpu().numpy()

    results = []
    for prob in probabilities:
        result = {
            'negative': prob[0],
            'positive': prob[1],
            'predicted_sentiment': 'positive' if prob[1] > prob[0] else 'negative'
        }
        results.append(result)

    return results

In [6]:
db = df[['id', 'POI', 'rating', 'recensione']]

In [7]:
feel_it_scores = []
for sentence in db['recensione']:
    feel_it_scores.append(get_sentiment_probabilities([sentence])[0])


In [8]:
negative = [score['negative'] for score in feel_it_scores]
positive = [score['positive'] for score in feel_it_scores]
FEEL_IT  = [score['predicted_sentiment'] for score in feel_it_scores]

In [13]:
db['negative'] = negative
db['positive'] = positive
db['FEEL_IT'] = FEEL_IT
df['FEEL_IT'] = FEEL_IT

In [14]:
db

,Unnamed: 0,id,POI,rating,recensione,FEEL_IT,negative,positive
0,0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume...",positive,0.000246,0.999754
1,1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...,positive,0.000248,0.999752
2,2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale è ...,positive,0.002336,0.997664
3,3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...,positive,0.336789,0.663211
4,4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta é il principale mo...,positive,0.000375,0.999625
...,...,...,...,...,...,...,...,...
5330,5330,663c8b16ec1e0d04a559c75e,Cacio Pepe e,5,Ci siamo trovati di passaggio dopo aver visita...,positive,0.000297,0.999703
5331,5331,663c8b16ec1e0d04a559c75f,Annette Jolie,5,"Girando per Assisi ,ho trovato un negozio fina...",positive,0.000226,0.999774
5332,5332,663c8b16ec1e0d04a559c75f,Annette Jolie,5,A pochi passi da Piazza del Comune abbiamo sco...,positive,0.000220,0.999780
5333,5333,663c8b16ec1e0d04a559c760,Sorella Luna,5,Negozio posto in una via secondaria ma dove va...,positive,0.000239,0.999761


In [15]:
# Salvo i risultati in un file CSV
db.to_csv("../sentiment/FEEL_IT_results.csv")

In [16]:
# Salvo i risultati in un file Excel
db.to_excel("../sentiment/FEEL_IT.xlsx", index=False)

In [17]:
# elimino dal df principale le colonne inutili
df.drop(['negative', 'neutral', 'positive'], axis=1, inplace=True)

In [18]:
# scampio l'ordine delle colonne
df = df[['id', 'POI', 'rating', 'recensione', 'GS', 'BERT', 'FEEL_IT']]

KeyError: "['GS'] not in index"

In [89]:
# Salvo il DataFrame in un file Excel
df.to_excel('../sentiment/sentiment_comparison.xlsx', index=False)

### Calcolo dell'Accuracy

In [90]:
db['GS']=0
db.loc[df['rating'] >= 3, 'GS']  = 'positive'
db.loc[df['rating'] <= 2, 'GS']  = 'negative'
db

,Unnamed: 0,id,POI,rating,recensione,FEEL_IT,GS
0,0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume...",positive,positive
1,1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...,positive,positive
2,2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale è ...,positive,positive
3,3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...,positive,positive
4,4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta é il principale mo...,positive,positive
...,...,...,...,...,...,...,...
5330,5330,663c8b16ec1e0d04a559c75e,Cacio Pepe e,5,Ci siamo trovati di passaggio dopo aver visita...,positive,positive
5331,5331,663c8b16ec1e0d04a559c75f,Annette Jolie,5,"Girando per Assisi ,ho trovato un negozio fina...",positive,positive
5332,5332,663c8b16ec1e0d04a559c75f,Annette Jolie,5,A pochi passi da Piazza del Comune abbiamo sco...,positive,positive
5333,5333,663c8b16ec1e0d04a559c760,Sorella Luna,5,Negozio posto in una via secondaria ma dove va...,positive,positive


In [91]:
print(classification_report(db['GS'], db['FEEL_IT']))

              precision    recall  f1-score   support

    negative       0.42      0.90      0.58       437
    positive       0.99      0.89      0.94      4898

    accuracy                           0.89      5335
   macro avg       0.71      0.90      0.76      5335
weighted avg       0.94      0.89      0.91      5335



In [92]:
# Calcola il classification report
report = classification_report(db['GS'], db['FEEL_IT'], output_dict=True)

# Creare un DataFrame pandas dal classification report
db_report = pd.DataFrame(report).transpose()

# Salvare il DataFrame come file Excel
excel_filename = '../sentiment/FEEL_IT_classification_report.xlsx'
db_report.to_excel(excel_filename, index=True)

## 3) MULTILINGUAL: bert-base-multilingual-uncased-sentiment
https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment 

Si tratta di un modello bert-base-multilingue-uncased ottimizzato per l'analisi del sentiment sulle recensioni dei prodotti in sei lingue: inglese, olandese, tedesco, francese, spagnolo e **italiano**. Prevede il sentiment della recensione sotto forma di numero di stelle (tra 1 e 5).

In [95]:
# Load model directly from the Hugging Face Hub
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
multilingual_model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment").to("cuda")

In [96]:
# Crea colonne per memorizzare le probabilità
df['1'] = 0
df['2'] = 0
df['3'] = 0
df['4'] = 0
df['5'] = 0

    # Itera attraverso ogni riga del DataFrame
for index, row in tqdm(df.iterrows()):
    sentence = row.recensione
        
    input_ids = tokenizer.encode(sentence, add_special_tokens=True, truncation=True, max_length=512)
        
    # Create tensor, use .cuda() to transfer the tensor to GPU
    tensor = torch.tensor(input_ids).long()
    # Fake batch dimension
    tensor = tensor.unsqueeze(0).to("cuda")

    # Call the model and get the logits
    logits = multilingual_model(tensor)
    # Remove the fake batch dimension
    logits = logits.logits.squeeze(0)

    # The model was trained with a Log Likelyhood + Softmax combined loss, hence to extract probabilities we need a softmax on top of the logits tensor
    proba = nn.functional.softmax(logits, dim=0)

    # Aggiungi le probabilità alle colonne appropriate
    proba = proba.to("cpu")
    df.loc[index, '1'] = proba[0].item()
    df.loc[index, '2'] = proba[1].item()
    df.loc[index, '3'] = proba[2].item()
    df.loc[index, '4'] = proba[3].item()
    df.loc[index, '5'] = proba[4].item()

# Calcola il sentimento prevalente
df['MULTILINGUAL'] = df[['1', '2', '3', '4', '5']].idxmax(axis=1)
db = df[['id', 'POI', 'rating', 'recensione', 'GS', '1', '2', '3', '4', '5', 'MULTILINGUAL']]
df.drop(columns=['1', '2', '3', '4', '5'], inplace=True)

# Tentare di convertire i valori della colonna 'MULTILINGUAL' in numerici, ignorando gli errori
df['sentiment_pred_numeric'] = pd.to_numeric(df['MULTILINGUAL'], errors='coerce')

# Applicare le trasformazioni solo ai valori numerici
df.loc[df['sentiment_pred_numeric'] >= 4, 'MULTILINGUAL'] = 'positive'
df.loc[df['sentiment_pred_numeric'] <= 2, 'MULTILINGUAL'] = 'negative'
df.loc[df['sentiment_pred_numeric'] == 3, 'MULTILINGUAL'] = 'neutral'

# Rimuovere la colonna temporanea utilizzata per la conversione
df.drop(columns=['sentiment_pred_numeric'], inplace=True)

C:\Users\franc\AppData\Local\Temp\ipykernel_14004\3973190333.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['1'] = 0
C:\Users\franc\AppData\Local\Temp\ipykernel_14004\3973190333.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['2'] = 0
C:\Users\franc\AppData\Local\Temp\ipykernel_14004\3973190333.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

In [130]:
df

,id,POI,rating,recensione,GS,BERT,FEEL_IT,MULTILINGUAL
0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume...",positive,positive,positive,positive
1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...,positive,positive,positive,positive
2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale è ...,positive,positive,positive,neutral
3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...,positive,positive,positive,positive
4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta é il principale mo...,positive,positive,positive,positive
...,...,...,...,...,...,...,...,...
5330,663c8b16ec1e0d04a559c75e,Cacio Pepe e,5,Ci siamo trovati di passaggio dopo aver visita...,positive,positive,positive,positive
5331,663c8b16ec1e0d04a559c75f,Annette Jolie,5,"Girando per Assisi ,ho trovato un negozio fina...",positive,positive,positive,positive
5332,663c8b16ec1e0d04a559c75f,Annette Jolie,5,A pochi passi da Piazza del Comune abbiamo sco...,positive,positive,positive,positive
5333,663c8b16ec1e0d04a559c760,Sorella Luna,5,Negozio posto in una via secondaria ma dove va...,positive,positive,positive,positive


In [131]:
db

,Unnamed: 0,id,POI,rating,recensione,GS,1,2,3,4,5,MULTILINGUAL
0,0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume...",positive,0.004849,0.009446,0.063394,0.392164,0.530147,positive
1,1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...,positive,0.001022,0.001809,0.033151,0.438019,0.525999,positive
2,2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale è ...,positive,0.004407,0.063242,0.514002,0.385526,0.032823,neutral
3,3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...,positive,0.010631,0.089289,0.340011,0.504914,0.055155,positive
4,4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta é il principale mo...,positive,0.002611,0.004687,0.043096,0.394785,0.554821,positive
...,...,...,...,...,...,...,...,...,...,...,...,...
5330,5330,663c8b16ec1e0d04a559c75e,Cacio Pepe e,5,Ci siamo trovati di passaggio dopo aver visita...,positive,0.001865,0.002672,0.008780,0.132900,0.853783,positive
5331,5331,663c8b16ec1e0d04a559c75f,Annette Jolie,5,"Girando per Assisi ,ho trovato un negozio fina...",positive,0.127622,0.136910,0.161904,0.224093,0.349471,positive
5332,5332,663c8b16ec1e0d04a559c75f,Annette Jolie,5,A pochi passi da Piazza del Comune abbiamo sco...,positive,0.003115,0.006855,0.028588,0.260003,0.701439,positive
5333,5333,663c8b16ec1e0d04a559c760,Sorella Luna,5,Negozio posto in una via secondaria ma dove va...,positive,0.014572,0.037150,0.088022,0.334005,0.526252,positive


In [132]:
# Salvo i risultati in un file CSV
db.to_csv("../sentiment/MULTILINGUAL_results.csv")
db.to_excel("../sentiment/MULTILINGUAL.xlsx", index=False)

In [100]:
# Salvo il DataFrame in un file Excel
df.to_excel('../sentiment/sentiment_comparison.xlsx', index=False)

### Calcolo dell'Accuracy

In [101]:
db = pd.read_csv('../sentiment/MULTILINGUAL_results.csv')

In [103]:
# converto i valori di 'MULTILINGUAL' in numerici
db['MULTILINGUAL'] = db['MULTILINGUAL'].map({5: 'positive', 4: 'positive', 3: 'neutral', 1: 'negative', 2: 'negative'})

In [104]:
db

,Unnamed: 0,id,POI,rating,recensione,GS,1,2,3,4,5,MULTILINGUAL
0,0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume...",positive,0.004849,0.009446,0.063394,0.392164,0.530147,positive
1,1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...,positive,0.001022,0.001809,0.033151,0.438019,0.525999,positive
2,2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale è ...,positive,0.004407,0.063242,0.514002,0.385526,0.032823,neutral
3,3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...,positive,0.010631,0.089289,0.340011,0.504914,0.055155,positive
4,4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta é il principale mo...,positive,0.002611,0.004687,0.043096,0.394785,0.554821,positive
...,...,...,...,...,...,...,...,...,...,...,...,...
5330,5330,663c8b16ec1e0d04a559c75e,Cacio Pepe e,5,Ci siamo trovati di passaggio dopo aver visita...,positive,0.001865,0.002672,0.008780,0.132900,0.853783,positive
5331,5331,663c8b16ec1e0d04a559c75f,Annette Jolie,5,"Girando per Assisi ,ho trovato un negozio fina...",positive,0.127622,0.136910,0.161904,0.224093,0.349471,positive
5332,5332,663c8b16ec1e0d04a559c75f,Annette Jolie,5,A pochi passi da Piazza del Comune abbiamo sco...,positive,0.003115,0.006855,0.028588,0.260003,0.701439,positive
5333,5333,663c8b16ec1e0d04a559c760,Sorella Luna,5,Negozio posto in una via secondaria ma dove va...,positive,0.014572,0.037150,0.088022,0.334005,0.526252,positive


In [105]:
db.to_csv("../sentiment/MULTILINGUAL_results.csv", index=False)

In [106]:
print(classification_report(db['GS'], db['MULTILINGUAL']))

              precision    recall  f1-score   support

    negative       0.63      0.75      0.69       437
     neutral       0.39      0.28      0.33       710
    positive       0.89      0.92      0.91      4188

    accuracy                           0.82      5335
   macro avg       0.64      0.65      0.64      5335
weighted avg       0.81      0.82      0.81      5335



In [107]:
# Creare un DataFrame pandas dal classification report
db_report = pd.DataFrame(report).transpose()

# Salvare il DataFrame come file Excel
excel_filename = '../sentiment/MULTILINGUAL_classification_report.xlsx'
db_report.to_excel(excel_filename, index=True)

## BMA

Bayesian Model Averaging (BMA):

**Descrizione**: Una tecnica statistica utilizzata per migliorare le previsioni o le inferenze combinando diversi modelli. Anziché scegliere un singolo modello, BMA prende in considerazione molti modelli possibili e pondera le loro previsioni in base alla probabilità che ciascun modello sia corretto.

**Applicazioni**: Utilizzata in varie discipline tra cui economia, epidemiologia, e machine learning per ottenere previsioni più robuste.

**Principi di BMA**: 
- **Incertezza del Modello**: BMA riconosce che non c'è un singolo modello che sia perfetto per rappresentare i dati. Invece di selezionare un solo modello, considera una gamma di modelli possibili.
- **Pesi Bayesiani**: Ogni modello è ponderato in base alla sua probabilità a posteriori, che riflette quanto bene il modello si adatta ai dati e quanto è plausibile a priori.
- **Combina Previsioni**: Le previsioni finali sono ottenute combinando le previsioni di tutti i modelli ponderate per la loro probabilità a posteriori.

**Vantaggi di BMA:**
- **Robustezza**: Migliora la robustezza delle previsioni riducendo il rischio di fare affidamento su un singolo modello che potrebbe essere mal specificato.
- **Incorpora Incertezza**: Fornisce una misura più realistica dell'incertezza nelle previsioni incorporando la variabilità tra modelli.

In [1]:
from bma_ import bma_test

In [4]:
bert = pd.read_excel('../sentiment/BERT.xlsx')
feel_it = pd.read_excel('../sentiment/FEEL_IT.xlsx')
multilingual = pd.read_excel('../sentiment/MULTILINGUAL.xlsx')
df = pd.read_excel('../sentiment/sentiment_comparison.xlsx')

In [5]:
# normalizzo i valori senza il neutrale
df['GS_norm'] = df['rating'].apply(lambda x: 'positive' if x >= 3 else 'negative')  


bert['positive_norm'] = bert['positive']/(bert['positive'] + bert['negative'])
bert['negative_norm'] = bert['negative']/(bert['positive'] + bert['negative'])
bert['BERT_norm'] = bert[['negative_norm', 'positive_norm']].idxmax(axis=1)
bert['BERT_norm'] = bert['BERT_norm'].apply(lambda x: 'positive' if x == 'positive_norm' else 'negative')
bert['GS_norm'] = df['GS_norm']

multilingual['positive_norm'] = (multilingual['5'] + multilingual['4'])/(multilingual['5'] + multilingual['4'] + multilingual['1'] + multilingual['2'])
multilingual['negative_norm'] = (multilingual['1'] + multilingual['2'])/(multilingual['5'] + multilingual['4'] + multilingual['1'] + multilingual['2'])
multilingual['MULTILINGUAL_norm'] = multilingual[['negative_norm', 'positive_norm']].idxmax(axis=1)
multilingual['MULTILINGUAL_norm'] = multilingual['MULTILINGUAL_norm'].apply(lambda x: 'positive' if x == 'positive_norm' else 'negative')
multilingual['GS_norm'] = df['GS_norm']

feel_it['GS_norm'] = df['GS_norm']

In [6]:
bert.head()

,id,POI,rating,recensione,negative,neutral,positive,BERT,positive_norm,negative_norm,BERT_norm,GS_norm
0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume...",0.001215,0.010892,0.987893,positive,0.998772,0.001228,positive,positive
1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...,0.000190,0.004603,0.995207,positive,0.999809,0.000191,positive,positive
2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale è ...,0.000429,0.001524,0.998047,positive,0.999571,0.000429,positive,positive
3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...,0.001172,0.004624,0.994203,positive,0.998822,0.001178,positive,positive
4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta é il principale mo...,0.000578,0.051220,0.948202,positive,0.999391,0.000609,positive,positive


In [7]:
bert.to_excel('../sentiment/BERT.xlsx', index=False)
multilingual.to_excel('../sentiment/MULTILINGUAL.xlsx', index=False)

I report generati dalle funzioni classification_report di scikit-learn forniscono una serie di metriche di valutazione per i modelli di classificazione, che includono precisione, recall, e F1-score per ciascuna classe, così come alcune metriche aggregate.

In [8]:
report1 = classification_report(df['GS_norm'], bert['BERT_norm'], output_dict=True)
report2 = classification_report(df['GS_norm'], feel_it['FEEL_IT'], output_dict=True)
report3 = classification_report(df['GS_norm'], multilingual['MULTILINGUAL_norm'], output_dict=True)

In [9]:
report1

{'negative': {'precision': 0.5683333333333334,
  'recall': 0.7803203661327232,
  'f1-score': 0.6576663452266153,
  'support': 437},
 'positive': {'precision': 0.9797254487856388,
  'recall': 0.9471212739893834,
  'f1-score': 0.9631475137548011,
  'support': 4898},
 'accuracy': 0.9334582942830365,
 'macro avg': {'precision': 0.7740293910594861,
  'recall': 0.8637208200610533,
  'f1-score': 0.8104069294907081,
  'support': 5335},
 'weighted avg': {'precision': 0.9460275379229102,
  'recall': 0.9334582942830365,
  'f1-score': 0.9381249700534297,
  'support': 5335}}

**Accuracy**
L'accuracy misura la percentuale di previsioni corrette sul totale delle previsioni. È semplice da interpretare, ma non è sempre il miglior indicatore delle prestazioni di un modello, specialmente in presenza di classi sbilanciate. Se le classi sono bilanciate, l'accuracy può essere una buona misura.

**F1-score**
L'F1-score è la media armonica della precisione e del recall. È particolarmente utile quando hai classi sbilanciate perché tiene conto sia dei falsi positivi che dei falsi negativi. Se il tuo dataset ha una distribuzione delle classi sbilanciata, l'F1-score è generalmente una scelta migliore rispetto all'accuracy.

In [10]:
# Estraggo l'affidabilità (ad esempio, F1-score) per ciascun modello e classe
reliabilities = {
    'bert': {label: {'f1-score': report1[label]['f1-score']} for label in ['negative', 'positive']},
    'feel_it': {label: {'f1-score': report2[label]['f1-score']} for label in ['negative', 'positive']},
    'multilingual': {label: {'f1-score': report3[label]['f1-score']} for label in ['negative', 'positive']}
}

In [11]:
reliabilities

{'bert': {'negative': {'f1-score': 0.6576663452266153},
  'positive': {'f1-score': 0.9631475137548011}},
 'feel_it': {'negative': {'f1-score': 0.5753823743627094},
  'positive': {'f1-score': 0.9372915994406799}},
 'multilingual': {'negative': {'f1-score': 0.6346483704974271},
  'positive': {'f1-score': 0.9551767676767676}}}

In [12]:
# Prevedo il sentiment delle recensioni utilizzando ciascun modello:

# Creo un dizionario di DataFrame
probs_df = {
    'bert': bert[['negative_norm', 'positive_norm']],
    'feel_it': feel_it[['negative', 'positive']],
    'multilingual': multilingual[['negative_norm', 'positive_norm']]
}

# Converto i DataFrame in una struttura di liste di probabilità
def convert_to_probs_list(df):
    return df.values.tolist()

# Creo il dizionario di probabilità nel formato richiesto
probs = {
    'bert': convert_to_probs_list(probs_df['bert']),
    'feel_it': convert_to_probs_list(probs_df['feel_it']),
    'multilingual': convert_to_probs_list(probs_df['multilingual'])
}

In [13]:
probs

{'bert': [[0.0012282729193036966, 0.9987717270806963],
  [0.00019073512877506847, 0.9998092648712249],
  [0.0004292438427276344, 0.9995707561572723],
  [0.0011778100590672752, 0.9988221899409327],
  [0.0006087639453282164, 0.9993912360546718],
  [0.024444569657522886, 0.9755554303424772],
  [0.047595417626451544, 0.9524045823735484],
  [0.04694044801967276, 0.9530595519803272],
  [0.05359915628239998, 0.9464008437176],
  [0.00013327557085171396, 0.9998667244291483],
  [0.00017306591388137744, 0.9998269340861187],
  [0.0002520420225903701, 0.9997479579774097],
  [0.00013062771848958792, 0.9998693722815104],
  [0.00018669922584910108, 0.9998133007741509],
  [0.00031028880288421867, 0.9996897111971158],
  [0.00010034627757815529, 0.9998996537224218],
  [0.001119153313825958, 0.998880846686174],
  [0.0003832247585497519, 0.9996167752414502],
  [0.004744533079909312, 0.9952554669200907],
  [0.022821024591936648, 0.9771789754080633],
  [0.0001416113332717835, 0.9998583886667283],
  [0.039235

In [14]:
# Applico il BMA per combinare le probabilità di ciascun modello
classes = {
    'bert': ['negative', 'positive'],
    'feel_it': ['negative', 'positive'],
    'multilingual': ['negative', 'positive']
}
model_list = ['bert', 'feel_it', 'multilingual']
source_tokens = df['recensione'].str.split().tolist()



In [15]:
len(source_tokens)

5335

In [16]:
source_tokens

[['Un',
  'palazzo',
  'appena',
  'restaurato,',
  'una',
  'ricca',
  'documentazione',
  'in',
  'ogni',
  'stanza',
  'e',
  "un'app",
  'per',
  'aiutare',
  'ancora',
  'di',
  'più.',
  'Non',
  'mancano',
  'gli',
  'affreschi,',
  'le',
  'documentazioni',
  'scientifiche',
  '...',
  'insomma',
  'è',
  'da',
  'non',
  'perdere'],
 ['Cinquecentesca',
  'dimora',
  'di',
  'una',
  'tra',
  'le',
  'famiglie',
  'illustri',
  'umbro',
  '-',
  'romane',
  '.',
  'Molto',
  'ben',
  'organizzati',
  'con',
  'ologrammi',
  'e',
  'proiezioni',
  'che',
  'guidano',
  "all'interno",
  'e',
  'in',
  'più',
  'noi',
  'abbiamo',
  'avuto',
  "l'occasione",
  'di',
  'avere',
  "un'oretta",
  'come',
  'guida',
  'il',
  'giovane',
  'Simone',
  '.',
  'Visita',
  'interessante',
  'e',
  'consigliata',
  '.'],
 ["L'imponente",
  'palazzo',
  'in',
  'stile',
  'rinascimentale',
  'è',
  'di',
  'per',
  'sé',
  'molto',
  'bello',
  'sia',
  'dal',
  'punto',
  'di',
  'vista',


In [17]:
results = bma_test(probs, reliabilities, model_list, classes, source_tokens)

100%|██████████| 5335/5335 [00:00<00:00, 186293.23it/s]


In [18]:
results

[('positive', [0.0037314704062768713, 0.9962685295937231]),
 ('positive', [0.0007448425734624609, 0.9992551574265375]),
 ('positive', [0.032016513938483544, 0.9679834860615164]),
 ('positive', [0.10825245761918964, 0.8917475423808103]),
 ('positive', [0.0019126892606127844, 0.9980873107393872]),
 ('positive', [0.01599558530412619, 0.9840044146958737]),
 ('positive', [0.025744191162050928, 0.9742558088379492]),
 ('positive', [0.3738420057821004, 0.6261579942178996]),
 ('positive', [0.36477931078681713, 0.6352206892131829]),
 ('positive', [0.0006548358157695475, 0.9993451641842304]),
 ('positive', [0.0007106223422534241, 0.9992893776577466]),
 ('positive', [0.0004745979751098859, 0.9995254020248902]),
 ('positive', [0.0012050400130603556, 0.9987949599869397]),
 ('positive', [0.0007682717683588246, 0.9992317282316412]),
 ('positive', [0.0025312471361615533, 0.9974687528638384]),
 ('positive', [0.0033235893394452406, 0.9966764106605548]),
 ('positive', [0.005860125064633009, 0.994139874935

In [20]:
# Inizializo le liste per memorizzare i risultati
pred_labels = []
prob_distributions = []

In [21]:
# Popolo le liste con i risultati della funzione bma_test
for predicted_label, prob_distribution in results:
    pred_labels.append(predicted_label)
    prob_distributions.append(prob_distribution)

In [22]:
# Aggiungo le nuove colonne al DataFrame
df["pred_BMA"] = pred_labels
df["prob_BMA"] = prob_distributions  

In [23]:
df

,id,POI,rating,recensione,GS,BERT,FEEL_IT,MULTILINGUAL,GS_norm,pred_BMA,prob_BMA
0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume...",positive,positive,positive,positive,positive,positive,"[0.0037314704062768713, 0.9962685295937231]"
1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...,positive,positive,positive,positive,positive,positive,"[0.0007448425734624609, 0.9992551574265375]"
2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale è ...,positive,positive,positive,neutral,positive,positive,"[0.032016513938483544, 0.9679834860615164]"
3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...,positive,positive,positive,positive,positive,positive,"[0.10825245761918964, 0.8917475423808103]"
4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta é il principale mo...,positive,positive,positive,positive,positive,positive,"[0.0019126892606127844, 0.9980873107393872]"
...,...,...,...,...,...,...,...,...,...,...,...
5330,663c8b16ec1e0d04a559c75e,Cacio Pepe e,5,Ci siamo trovati di passaggio dopo aver visita...,positive,positive,positive,positive,positive,positive,"[0.001125514444687801, 0.9988744855553121]"
5331,663c8b16ec1e0d04a559c75f,Annette Jolie,5,"Girando per Assisi ,ho trovato un negozio fina...",positive,positive,positive,positive,positive,positive,"[0.07282959418434247, 0.9271704058156576]"
5332,663c8b16ec1e0d04a559c75f,Annette Jolie,5,A pochi passi da Piazza del Comune abbiamo sco...,positive,positive,positive,positive,positive,positive,"[0.0023589594660805057, 0.9976410405339194]"
5333,663c8b16ec1e0d04a559c760,Sorella Luna,5,Negozio posto in una via secondaria ma dove va...,positive,positive,positive,positive,positive,positive,"[0.012819275241193633, 0.9871807247588065]"


In [24]:
df.to_excel('../sentiment/sentiment_comparison.xlsx', index=False)

# Risultati post BMA

In [25]:
df = pd.read_excel('../sentiment/sentiment_comparison.xlsx')

In [26]:
df

,id,POI,rating,recensione,GS,BERT,FEEL_IT,MULTILINGUAL,GS_norm,pred_BMA,prob_BMA
0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume...",positive,positive,positive,positive,positive,positive,"[0.0037314704062768713, 0.9962685295937231]"
1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...,positive,positive,positive,positive,positive,positive,"[0.0007448425734624609, 0.9992551574265375]"
2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale è ...,positive,positive,positive,neutral,positive,positive,"[0.032016513938483544, 0.9679834860615164]"
3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...,positive,positive,positive,positive,positive,positive,"[0.10825245761918964, 0.8917475423808103]"
4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta é il principale mo...,positive,positive,positive,positive,positive,positive,"[0.0019126892606127844, 0.9980873107393872]"
...,...,...,...,...,...,...,...,...,...,...,...
5330,663c8b16ec1e0d04a559c75e,Cacio Pepe e,5,Ci siamo trovati di passaggio dopo aver visita...,positive,positive,positive,positive,positive,positive,"[0.001125514444687801, 0.9988744855553121]"
5331,663c8b16ec1e0d04a559c75f,Annette Jolie,5,"Girando per Assisi ,ho trovato un negozio fina...",positive,positive,positive,positive,positive,positive,"[0.07282959418434247, 0.9271704058156576]"
5332,663c8b16ec1e0d04a559c75f,Annette Jolie,5,A pochi passi da Piazza del Comune abbiamo sco...,positive,positive,positive,positive,positive,positive,"[0.0023589594660805057, 0.9976410405339194]"
5333,663c8b16ec1e0d04a559c760,Sorella Luna,5,Negozio posto in una via secondaria ma dove va...,positive,positive,positive,positive,positive,positive,"[0.012819275241193633, 0.9871807247588065]"


In [27]:
print(classification_report(df['GS_norm'], df['pred_BMA']))

              precision    recall  f1-score   support

    negative       0.60      0.84      0.70       437
    positive       0.98      0.95      0.97      4898

    accuracy                           0.94      5335
   macro avg       0.79      0.89      0.83      5335
weighted avg       0.95      0.94      0.95      5335



In [28]:
df['pred_BMA'].value_counts()

pred_BMA
positive    4731
negative     604
Name: count, dtype: int64

In [29]:
df['GS_norm'].value_counts()

GS_norm
positive    4898
negative     437
Name: count, dtype: int64

In [35]:
# Filtro il DataFrame dove 'GS_norm' è 'positive' e 'pred_BMA' è 'negative'
rec_positive = df[(df['GS_norm'] == 'positive') & (df['pred_BMA'] == 'negative')]

# Seleziono solo le colonne 'recensione' e 'prob_BMA'
rec_positive_to_save = rec_positive[['recensione', 'prob_BMA']]

In [36]:
# Salvo le frasi in un file Excel
rec_positive_to_save.to_excel('../sentiment/rec_pos_to_neg.xlsx', index=False)

In [37]:
# Filtra il DataFrame dove 'GS_norm' è 'positive' e 'pred_BMA' è 'negative'
rec_positive = df[(df['GS_norm'] == 'positive') & (df['pred_BMA'] == 'negative')]

# Seleziono solo le colonne 'recensione' e 'prob_BMA'
rec_negative_to_save = rec_positive[['recensione', 'prob_BMA']]

# Salvo le frasi in un file Excel
rec_negative_to_save.to_excel('../sentiment/rec_neg_to_pos.xlsx', index=False)
